<a href="https://colab.research.google.com/github/mitramir55/Kaggle_NLP_competition/blob/master/Sequence_classification_with_Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [ ]:
!pip install transformers

In [4]:
from transformers import (TFRobertaForSequenceClassification,
                            RobertaTokenizer,)


In [7]:
roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base")
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
sentence = "babe you see me? I see you"
roberta_tokenizer.tokenize(sentence)

['b', 'abe', 'Ġyou', 'Ġsee', 'Ġme', '?', 'ĠI', 'Ġsee', 'Ġyou']

In [11]:
# Bert Tokenizer for all of them
import tensorflow_hub as hub
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization
FullTokenizer = tokenization.FullTokenizer

BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
disc = 'Base_uncased'

bert_layer = hub.KerasLayer(BERT_MODEL_HUB, trainable=True)

print('Bert layer is ready to use!')

to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

tokenizer = FullTokenizer(vocabulary_file, to_lower_case)

print('Bert Tokenizer is ready!!!')


INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'.
INFO:absl:Downloaded https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2, Total size: 421.50MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'.


Bert layer is ready to use!
Bert Tokenizer is ready!!!


In [12]:
tokenizer.tokenize("Systolic arrays are cool. This 🐳 is cool too.")

['sy',
 '##sto',
 '##lic',
 'arrays',
 'are',
 'cool',
 '.',
 'this',
 '[UNK]',
 'is',
 'cool',
 'too',
 '.']

In [ ]:
['sy','##sto','##lic','arrays','are','cool','.','this','[UNK]', 'is','cool','too','.']

In [13]:
import tensorflow_datasets
data = tensorflow_datasets.load("glue/mrpc")

train_dataset = data["train"]
validation_dataset = data["validation"]

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/mrpc/1.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.193dfb384738416eada1fef87e6ddb61...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.a5898b1316594ab49c6408ae78ee7c80...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.93452e29c57e4b588c6e4ab1a0d0a542...
/usr/local/lib/python3.6/dist-packages/urllib3/co

Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-train.tfrecord. Shard lengths: [3668]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-validation.tfrecord. Shard lengths: [408]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteSP36ZS/glue-test.tfrecord. Shard lengths: [1725]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.


In [19]:
example = list(train_dataset.__iter__())[2]
print('',
    'idx:      ', example['idx'],       '\n',
    'label:    ', example['label'],     '\n',
    'sentence1:', example['sentence1'], '\n',
    'sentence2:', example['sentence2'],
)

 idx:       tf.Tensor(3017, shape=(), dtype=int32) 
 label:     tf.Tensor(1, shape=(), dtype=int64) 
 sentence1: tf.Tensor(b'Spider-Man snatched $ 114.7 million in its debut last year and went on to capture $ 403.7 million .', shape=(), dtype=string) 
 sentence2: tf.Tensor(b'Spider-Man , rated PG-13 , snatched $ 114.7 million in its first weekend and went on to take in $ 403.7 million .', shape=(), dtype=string)


In [21]:
seq0 = example['sentence1'].numpy().decode('utf-8')  # Obtain bytes from tensor and convert it to a string
seq1 = example['sentence2'].numpy().decode('utf-8')  # Obtain bytes from tensor and convert it to a string

print("First sequence:", seq0)
print("Second sequence:", seq1)

First sequence: Spider-Man snatched $ 114.7 million in its debut last year and went on to capture $ 403.7 million .
Second sequence: Spider-Man , rated PG-13 , snatched $ 114.7 million in its first weekend and went on to take in $ 403.7 million .


In [24]:
encoded_roberta_sequence = roberta_tokenizer.encode(seq0, seq1,max_length=128)

In [ ]:
roberta_special_tokens = [roberta_tokenizer.sep_token_id, roberta_tokenizer.cls_token_id]

In [ ]:
encoded_roberta_sequence

In [29]:
from transformers import glue_convert_examples_to_features
def token_type_ids_removal(example, label):
    del example["token_type_ids"]
    return example, label


In [32]:

roberta_train_dataset = glue_convert_examples_to_features(train_dataset, roberta_tokenizer, 128, 'mrpc') # outputs examples and labels
#roberta_train_dataset = roberta_train_dataset.map(token_type_ids_removal)
roberta_train_dataset = roberta_train_dataset.shuffle(100).batch(32)

roberta_validation_dataset = glue_convert_examples_to_features(validation_dataset, roberta_tokenizer, 128, 'mrpc')
#roberta_validation_dataset = roberta_validation_dataset.map(token_type_ids_removal)
roberta_validation_dataset = roberta_validation_dataset.batch(64)


In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
roberta_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
roberta_history = roberta_model.fit(roberta_train_dataset, epochs=3, validation_data=roberta_validation_dataset)

Epoch 1/3


      6/Unknown - 220s 37s/step - loss: 0.6538 - accuracy: 0.6562

In [5]:
import time, psutil
uptime = time.time() - psutil.boot_time()
print('How much I used?\n {} hours, and {:.2f} minutes '.format(uptime//3600, uptime%60))
remain = 12*60*60 - uptime
print('How much is remaining?\n {} hours, and {:.2f} minutes '.format(remain//3600, remain%60))

How much I used?
 0.0 hours, and 4.81 minutes 
How much is remaining?
 11.0 hours, and 55.19 minutes 
